In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from  datascience import *

from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score
from random import choices
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors    import KNeighborsClassifier 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix, classification_report, precision_score, accuracy_score
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')


## Question 1: (Theoretical question)

We now review $k$-fold cross-validation.

(a) Explain how $k$-fold cross-validation is implemented in steps.

(b) What are the advantages and disadvantages of $k$-fold crossvalidation
relative to:

i. The validation set approach?

ii. LOOCV?

- a
    - k-fold cross validation works in a simular way to LOOCV how ever instead of having n-samples , we split our data up into k chuncks where each sample is comprised of test and traing data  
- b 
    - one advantange of k-fold relative to LOOCV is that it is way less computationally expensive due to the fact that we are not making a new model n times , we are taking k samples where k<n then using those samples to each train and test a model to approximate various parameters. The advantage over a validation set approach is that our approximations approach the accuracy of the estimates we get from LOOCV. A small disadvantage is that for each sample there will be slightly more bias relative to a sample from LOOCV because there is less training data.


# Question 2: (Programming question)

In previous chapter we used logistic regression to predict the probability of `default` using `income` and `balance` on the `Default` data set. Estimate the test error and Decision Boundary of this logistic regression model using the `validation set` approach. 

In [49]:
Default = pd.read_csv("Default.csv")
default = Default.values[:,1]
Balance = Default.values[:,3]
Income  = Default.values[:,4]
y       = Default.default.factorize()[0]
X       = Default.values[:,3:5]



(i). Do not forget to set a random seed before beginning your analysis. Split the sample set into a `training set` and a `validation set`.

(ii) Fit a logistic regression model that uses income and balance to predict default using only the training observations.

(iii). Obtain a prediction of default status for each individual in the validation set 

(iv). Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.

(v) State the Decision Boundary of the Logistic Regression: 

(vi) Plot the data with X Axis as Balance and Y Axis as Income. Put blue color for Non -Defaulters and Red colors for defaulters. Name the X Axis as Balance and Y Axis as Income. 

(vi). Generate 10 different validation set and training set and calculate the variability of the test error estimate across 10 different validation set. Remove doing the np.random.seed() in the loop. 

In [48]:
np.random.seed(2)
# split data
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.5)
# fit model
model = LogisticRegression(solver = 'newton-cg')
model.fit(X_train,y_train)
# prediction
y_pred = model.predict(X_train)
# compute error
score =  1- accuracy_score(y_test, y_pred)
score

0.04479999999999995

In [39]:
intercept = model.intercept_ 
slope1 = model.coef_[0,0]
slope2 = model.coef_[0,1]


print(str(intercept[0]) + '+' + str(slope1)+'x1' + ' + ' + str(slope2)+'x2' + ' = ' + '0' )

-11.06423194052087+0.005424658644462365x1 + 1.9838262810826255e-05x2 = 0


In [41]:
x = np.array([11.06423/model.coef_,11.06423/model.coef_,11.06423/model.coef_])
y = np.array([np.min(Income),np.max(Income),np.mean(Income)])

In [42]:
plt.plot(x,y)
plt.show()

ValueError: x and y can be no greater than 2-D, but have shapes (3, 1, 2) and (3,)

In [43]:
plt.scatter(X_train[y_train==0,0],X_train[y_train==0,1],color = 'blue')
plt.scatter(X_train[y_train==1,0],X_train[y_train==1,1],color = 'bred')
plt.plot(x,y,linewidth = 4, color = 'black')


ValueError: x and y can be no greater than 2-D, but have shapes (3, 1, 2) and (3,)

for some reason my plots arent showing here are the trouble shooing steps i followed:
- pip3 uninstall matplotlib
- pip3 install matplotlib
matplotlib-3.3.2 is the version i am using 
the plots above should yield the correct results
i tried to install an older version of matplotlib
- i did pip3 install matplotlib==3.2
so far i have had no luck 

In [52]:
test_error = make_array()

for repeat in np.arange(10):
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.5)
    model = LogisticRegression(solver = 'newton-cg')
    model.fit(X_train,y_train)
    y_pred = model.predict(X_train)
    score = 1- accuracy_score(y_test, y_pred)
    test_error = np.append(test_error, score)
np.std(test_error)  

0.0006399999999999954

## Question 3 : (Programming question)

We will now perform cross-validation on a simulated data set.

(a) Generate a simulated data set as follows:

In [53]:
np.random.seed(1)  # Random numbers generated by Python are different from those generated by R, as mentioned in Exercise 3.11
x = np.random.normal(size=100)
epsilon = np.random.normal(size=100)

y = x - 2 * x**2 + epsilon

In this data set, what is $n$ and what is $p$? Write out the model
used to generate the data in equation form.

(b) Create a scatterplot of $X$ against $Y$. Comment on what you find.


(c) Set a random seed, and then compute the LOOCV errors that
result from fitting the following four models using least squares:

$$\begin{align*}
i. Y &= β0 + β1X + \epsilon \\
ii. Y &= β0 + β1X + β2X^2 + \epsilon \\
iii. Y &= β0 + β1X + β2X^2 + β3X^3 + \epsilon \\
iv. Y &= β0 + β1X + β2X^2 + β3X^3 + β4X^4 + \epsilon. \\
\end{align*}$$

Note you may find it helpful to use the `data.frame()` function
to create a single data set containing both $X$ and $Y$ .


In [54]:
# Organize data into a dataframe (easier to handle)
df = pd.DataFrame({'x':x, 'y':y})

# Initiate variables
min_deg = 1  # Minimum degree of the polynomial equations considered
max_deg = 4+1  # Maximum degree of the polynomial equations considered

In [55]:
## method 1: 

# Set new random seed
np.random.seed(20)

model = LinearRegression()
loo = LeaveOneOut()
scores = np.array([])

# Compute mean squared error (MSE) for the different polynomial equations.
for i in range(min_deg, max_deg):
    # Leave-one-out cross validation 
    for train, test in loo.split(df):
        X_train = df['x'][train].to_numpy().reshape(-1,1)
        y_train = df['y'][train].to_numpy()
        X_test = df['x'][test].to_numpy().reshape(-1,1)
        y_test = df['y'][test].to_numpy()
        
        poly   = PolynomialFeatures(i) 
        X_train_poly = poly.fit_transform(X_train)
        X_test_poly = poly.fit_transform(X_test)
        model.fit(X_train_poly, y_train)
        y_pred = model.predict(X_test_poly)

        # MSE
        score = mean_squared_error(y_test, y_pred)
        scores= np.append(scores,score)
    print('Model %i (MSE): %f' % (i,np.mean(scores)))
    scores = []

Model 1 (MSE): 6.260764
Model 2 (MSE): 0.914290
Model 3 (MSE): 0.926877
Model 4 (MSE): 0.866912


In [61]:
## method 2, use the cross_val_score function directly
# Initialize the model
lm = LinearRegression()

# Create LOOCV object
loo = LeaveOneOut()
scores = np.array([])

for i in range(min_deg, max_deg):
    poly   = PolynomialFeatures(i) 
    X_poly = poly.fit_transform(x.reshape(-1,1))
    score  = cross_val_score(lm, X_poly, df.y, cv=5, scoring='neg_mean_squared_error').mean()
    scores = np.append(scores,score)

# The output is negative MSE
scores 


array([-6.46172316, -0.89727766, -0.93279051, -0.88994609])

In [62]:
lm = LinearRegression()
np.random.seed(3)
# Create LOOCV object
loo = LeaveOneOut()
scores2 = np.array([])

for i in range(min_deg, max_deg):
    poly   = PolynomialFeatures(i) 
    X_poly = poly.fit_transform(x.reshape(-1,1))
    score  = cross_val_score(lm, X_poly, df.y, cv=5, scoring='neg_mean_squared_error').mean()
    scores2 = np.append(scores2,score)

# The output is negative MSE
scores2 

array([-6.46172316, -0.89727766, -0.93279051, -0.88994609])

(d) Repeat (c) using another random seed, and report your results.
Are your results the same as what you got in (c)? Why?

(e) Which of the models in (c) had the smallest LOOCV error? Is
this what you expected? Explain your answer.


- d
    - after using a different random seed we get virtually the same results wich is not surprsing we are using LOOCV which mean that eventually the n sub samples will cover all the possible combonations of items being left out 
- e
    -  the models that have the smallest LOOCV error are the 4 degree polynomials but even after the second order polonomial is modeled we dont see a decrease the the MSE from 2-3 as we see from 1-2 . The fact that the higher order polynomials have smaller MSEs is not surprising as they are more flexable 

## Question 4: (Programming question)

We will now consider the `Boston` housing data set, the following code is used to import the `Boston` dataset.

In [63]:
from sklearn.datasets import load_boston
boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['MEDV'] = pd.Series(boston.target)
medv = df['MEDV']
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


(a) Based on this data set, provide an estimate for the population
mean of `medv`. Call this estimate $\hat{\mu}$.

(b) Provide an estimate of the standard error of $\hat{\mu}$. Interpret this
result.

Hint: We can compute the standard error of the sample mean by
dividing the sample standard deviation by the square root of the
number of observations.

(c) Now estimate the standard error of $\hat{\mu}$ using the bootstrap. How
does this compare to your answer from (b)?

In [64]:
B_times = 1000
n = len(df)
means = np.array([])
for i in range(B_times):
    x_bootstrap = choices(medv, k = n) ## draw samples with replacement
    means = np.append(means, np.mean(x_bootstrap))
np.std(means)    

0.402744400383397

In [67]:
sortedmeans = sorted(means)

(d) Based on your bootstrap estimate from (c), provide a 95% confidence
interval for the mean of `medv`. 

Hint: You can approximate a 95% confidence interval using the
formula $[\hat{mu} − 2SE(\hat{\mu}), \hat{\mu} + 2SE(\hat{\mu})]$.

(e) Based on this data set, provide an estimate, $\hat{\mu}_{med}$, for the median
value of `medv` in the population.

(f) We now would like to estimate the standard error of $\hat{\mu}_{med}$. Unfortunately,
there is no simple formula for computing the standard
error of the median. Instead, estimate the standard error of the
median using the bootstrap. Comment on your findings.

Hint: Similar as q(c)

(g) Based on this data set, provide an estimate for the tenth percentile
of `medv` in Boston suburbs. Call this quantity $\hat{\mu}_{0.1}$. (You
can use the `quantile()` function.)

(h) Use the bootstrap to estimate the standard error of $\hat{\mu}_{0.1}$. Comment
on your findings.

In [75]:
mu_95_CI_medv = [sortedmeans[25], sortedmeans[975]]
print('95% confidence interval for mdev')
mu_95_CI_medv

95% confidence interval for mdev


[21.735375494071146, 23.309090909090912]

In [76]:
sortedset = sorted(medv)
print('median for whole medv')
sortedset[int(len(sortedset)/2)]

median for whole medv


21.2

In [80]:
B_times = 1000
n = len(df)
medians = np.array([])
for i in range(B_times):
    x_bootstrap = choices(medv, k = n) ## draw samples with replacement
    x_bootsorted = sorted(x_bootstrap)
    medians = np.append(medians,x_bootsorted[int(len(x_bootsorted)/2)])
np.mean(medians)

21.199799999999996

the above value is the estimate for $\hat{\mu}_{med}$ and the value below is its standrard error.

In [87]:
np.std(medians)/np.power(n,.5)

0.017087311243539648

In [83]:
medvsorted = sorted(medv)
np.quantile(medvsorted,.01)

7.01

In [88]:
B_times = 1000
n = len(df)
tpcntl = np.array([])
for i in range(B_times):
    x_bootstrap = choices(medv, k = n) ## draw samples with replacement
    x_bootsorted = sorted(x_bootstrap)
    tpcntl = np.append(tpcntl,np.quantile(x_bootsorted,.01))
    
np.mean(tpcntl),np.std(tpcntl)/np.power(n,.5)

(6.9732650000000005, 0.025674171314335855)

for parts h and g i have printeed out the mean of the all the samples for that specific parameter along with its standard error.
- $\hat{\mu}_{med}$ = 21.199799 and the SE = 0.01708731
- $\hat{\mu}_{0.1}$ = 6.973265 and the SE = 0.0256741

## Question 5: (Theoretical  and programing question)

We will now derive the probability that a given observation is part
of a bootstrap sample. Suppose that we obtain a bootstrap sample
from a set of $n$ observations.

(a) What is the probability that the first bootstrap observation is
_not_ the $j$th observation from the original sample? Justify your
answer.

(b) What is the probability that the second bootstrap observation
is _not_ the $j$th observation from the original sample?

(c) Argue that the probability that the $j$th observation is not in the
bootstrap sample is $(1 − 1/n)^n$.

(d) When $n = 5$, what is the probability that the $j$th observation is
in the bootstrap sample?

(e) When $n = 100$, what is the probability that the $j$th observation
is in the bootstrap sample?

(f) When $n = 10, 000$, what is the probability that the $j$th observation
is in the bootstrap sample?

(g) Create a plot that displays, for each integer value of $n$ from 1
to 100, 000, the probability that the jth observation is in the
bootstrap sample. Comment on what you observe.


In [96]:
import numpy as np
import matplotlib.pyplot as plt

n = 100000   # Number of observations
prob = [1 - (1-1/n)**n for n in range(1, n)] # If the body of the loop is short, we can use this method
x = [n for n in range(1, n)]
 

fig, ax = plt.subplots(figsize = (10, 8))
ax.plot(x, prob)

### your analysis here:





- a 
    - the probability that the jth observation is not in the in the origonal sample is 1-1/n . when taking our first bootstrap sample each observaation has a equal probability, and becuase we are interested in the not case we then put 1-1/n.
- b 
    - so the second bootstrap sample has each observation with a probability of 1/n , it would be n-1 but there is replacement so the probability stays the same .
- c 
    - so for the first sample the P(j) to not be in the sample is 1-1/n, this can be applied to each redraw becuase there is replacement in these samples. because these selections are random and independent we can say that the P(j) is not in the nth sample is the same as multiplying all the prior probabilities, and since there are n samples we can get (1-1/n) ^N
- d
    - becuase we want the opposite probability of the equation we just described we can say 1-(1-1/n)^n is the probability the th observation is in the sample when n =5 the probability is = .67232 and the same process can be done with 100 and 10000 where when n = 100 the P(j is in the sample) = .633967 and when n=10000, the  P(j is in the sample ) = .632138
- g 
     - the graph shows that as our sample size increases the probability of j being in the sample approaches a hard limit somewhere above 0. this limit must be 1-1/e as this is approximately 1-p(observation isnt in sample ).